In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm

from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate
from llama_index.core.evaluation import SemanticSimilarityEvaluator
from llama_index.core.evaluation import CorrectnessEvaluator
from llama_index.core.evaluation import FaithfulnessEvaluator

from utils import json_load

## data prepare

In [2]:
# load ground-truth question and answer pair
dataset_dir = os.path.join('data', 'source')
dataset_file_path = os.path.join(dataset_dir, 'lee_course0_rag_dataset.csv')
df = pd.read_csv(dataset_file_path)
print(df.shape)
df.head(3)

(8, 5)


,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,這門課希望學生從使用者變成什麼？,['各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini)
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,['第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握A...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini)
2,哪個軟體被用來合成講師的聲音？,['也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini)


In [3]:
# load llm response
response_dir = os.path.join('data', 'source')
response_file_path = os.path.join(response_dir, 'baseline_rag_result_R3.json')
#response_file_path = os.path.join(response_dir, 'baseline_rag_result.json')
response = json_load(response_file_path)
response_df = pd.DataFrame(response)
df = pd.merge(df, response_df, how='left', on='query')
df.head(3)

load data from: data/source/baseline_rag_result_R3.json


,query,reference_contexts,reference_answer,reference_answer_by,query_by,response,retrieved_nodes,reranked_nodes
0,這門課希望學生從使用者變成什麼？,['各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini),這門課希望學生從使用者變成創作者。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'f1b8dce2-5b45-4ed8-b44d-0bf6e2306877'...
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,['第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握A...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini),抱歉，無法提供該工具的資訊。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...
2,哪個軟體被用來合成講師的聲音？,['也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini),沒有提到任何軟體被用來合成講師的聲音。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...


In [4]:
pipeline_name = os.path.splitext(os.path.basename(response_file_path))[0]
print(pipeline_name)
save_dir = os.path.join('data', 'temp')
os.makedirs(save_dir, exist_ok=True)
save_file_path = os.path.join(save_dir, f"{pipeline_name}_semantic.csv")

baseline_rag_result_R3


In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.evaluation import SemanticSimilarityEvaluator
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
semantic_evaluator = SemanticSimilarityEvaluator(embed_model=embed_model, similarity_threshold=0.8)

In [6]:
similarity_score = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    reference = row.reference_answer
    response = row.response
    result = await semantic_evaluator.aevaluate(
        response=response,
        reference=reference,
    )
    similarity_score.append(result.score)

df['similarity_score'] = similarity_score
print('write result to ' + save_file_path)
df.to_csv(save_file_path, index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.24it/s]

write result to data/temp/baseline_rag_result_R3_semantic.csv


In [7]:
df.head(3)

,query,reference_contexts,reference_answer,reference_answer_by,query_by,response,retrieved_nodes,reranked_nodes,similarity_score
0,這門課希望學生從使用者變成什麼？,['各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini),這門課希望學生從使用者變成創作者。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'f1b8dce2-5b45-4ed8-b44d-0bf6e2306877'...,0.468794
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,['第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握A...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini),抱歉，無法提供該工具的資訊。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,0.263029
2,哪個軟體被用來合成講師的聲音？,['也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini),沒有提到任何軟體被用來合成講師的聲音。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,0.222621
